In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from tqdm import tqdm

In [5]:
df_train = pd.read_csv("content/train.csv", index_col=0)
df_test = pd.read_csv("content/test.csv", index_col=0)
urls = set(df_test.index.str[:24]).union(set(df_train.index.str[:24]))

In [6]:
res_arr = []
for url in tqdm(urls):
    base = {'url': url, 'title': None, 'description': None, 'keywords': None, 'copyright': None,
            'article_categories': None, 'article_authors': None, 'article_publication_date': None,
            'article_word_count': None,
            'text': None,}
    try:
        code = requests.get(f'https://www.rbc.ru/politics/01/01/2021/{url}')
        html = BeautifulSoup(code.text, 'lxml')

        base["title"] = html.find('meta', attrs={'name': "title"})["content"]
        base["description"] = html.find('meta', attrs={'name': "description"})["content"]
        base["keywords"] = html.find('meta', attrs={'name': "keywords"})["content"]
        base["copyright"] = html.find('meta', attrs={'name': "copyright"})["content"]

        for script in html.find_all('script', type=''):
            if script.string and script.string.find('article_word_count') != -1:
                sc_text = script.string
                base["article_categories"] = re.search(r'article_categories: \[(.*?)],', sc_text).group(1)
                base["article_authors"] = re.search(r'article_authors: \[(.*?)],', sc_text).group(1)
                base["article_publication_date"] = re.search(r'article_publication_date: "(.*?)"', sc_text).group(1)
                base["article_word_count"] = re.search(r'article_word_count: "(.*?)"', sc_text).group(1)
                break

        text = []
        for div in html.find_all('div', attrs={'class': 'article__text'}):
            if div:
                for p in div.find_all('p'):
                    if p.get_text().strip():
                        text.append(p.get_text().strip())
        base["text"] = '|'.join(text)
    except Exception as e:
        print(e)
    res_arr.append(base)

100%|██████████| 9890/9890 [41:25<00:00,  3.98it/s]  


In [7]:
scraped = pd.DataFrame(res_arr)

In [10]:
scraped.to_csv("content/scraped.csv", line_terminator='\n', index=False)